In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
import sys
sys.path.append('../src')
import numpy as np
import matplotlib.pyplot as plt

from dem import dem4geoid

In [ ]:
dem = dem4geoid(bbox=[-4, 4, 2, 12], downloads_dir='downloads')
dem

In [ ]:
import os
# os.chdir('..')
# os.getcwd()

In [ ]:
dem['z'].plot(cmap='terrain')
plt.show()

In [ ]:
import os
import requests
from tqdm import tqdm

def download_srtm30plus(url=None, downloads_dir=None, bbox=None):
    '''
    Parameters
    ----------
    url           : url of the srtm30plus tile
    downloads_dir : directory to download the file to
    bbox          : bbox of the area of interest (to be provided if url is not provided)
    
    Returns
    -------
    fname         : Name of downloaded file
    '''

    if not url:
        url = fetch_url(bbox=bbox)
        
    filename = url.split('/')[-1]
    filepath = os.path.join(downloads_dir, filename) if downloads_dir else filename

    # Check if the file already exists
    if os.path.exists(filepath):
        # Get the expected size of the file
        response_head = requests.head(url, verify=False)
        total_size = int(response_head.headers.get('content-length', 0))

        # Check if the existing file size matches the expected size
        if os.path.getsize(filepath) == total_size:
            print(f'File {filename} already exists and is complete.')
            return filename
        else:
            print(f'File {filename} already exists but is incomplete. Redownloading...')

    if downloads_dir:
        os.makedirs(downloads_dir, exist_ok=True)
        print(f'Downloading {filename} to {downloads_dir} ...')
    else:
        print(f'Downloading {filename} to {os.getcwd()} ...')

    response = requests.get(url, verify=False, stream=True)
    total_size = int(response.headers.get('content-length', 0))

    with open(filepath, 'wb') as f, tqdm(
        desc=filename,
        total=total_size,
        unit='iB',
        unit_scale=True,
        unit_divisor=1024,
    ) as pbar:
        for data in response.iter_content(chunk_size=1024):
            size = f.write(data)
            pbar.update(size)

    return filename
